<a href="https://colab.research.google.com/github/abdulwaqar844/BATCH62_Q2_Assignment_01/blob/main/Project_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain pinecone pinecone-client google-generativeai openai tqdm -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 8.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import time
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key=userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

index_name = "langchain-rag-project"  # change if desired
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [3]:
pip install langchain_google_genai uuid langchain_pinecone langchain langchain_community pinecone -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.8 MB/s eta 0:00:00


In [4]:
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_core.documents import Document

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=userdata.get('GOOGLE_API_KEY')
    )
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['beb59e4d-8db6-47a2-9198-9720ebc067b5',
 'c65efa2d-cbcb-475e-aa53-a332b880da53',
 '5d5597f7-b072-45e7-a7b2-7d5d556ac446',
 '348be65b-806c-492e-ac0f-078ef8b07bad',
 '6ce3f1cd-0a48-49aa-b880-2bfbe2f2fd75',
 'ebf195d0-9d8f-4ab0-9c2d-a468f57d6557',
 '076f3922-3c68-4783-b29c-12f3ad39fa08',
 '55a56efa-4ced-4c6d-b8ac-641ecbb8ce8f',
 'e66d5764-38f8-409c-af38-0ad6c53e315e',
 '3a1dc74c-1a63-445b-be17-32ad4d9a8ea4']

In [20]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [6]:
# Step 3: Download the dataset and load it locally
import requests
from langchain.document_loaders import CSVLoader

url = 'https://github.com/JahanzaibTayyab/AI-201/raw/main/class06-20250105/rag/dataset/faq_bot_university.csv'
file_path = 'faq_bot_university.csv'

response = requests.get(url)
with open(file_path, 'wb') as f:
    f.write(response.content)
# changed line
loader = CSVLoader(file_path='faq_bot_university.csv', source_column='Answer')
documents = loader.load()

In [7]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['60493443-5535-4668-925b-abc0ac9e2693',
 '033fd3c4-5502-40d0-b81e-815c43dd23c0',
 '9c8ef2a1-1f1a-4848-becb-992ac544e555',
 'f1487231-37ee-4350-99b4-8170a2f60692',
 'd9a340b9-5a98-4477-85fa-cb4abdeef69d',
 '7752feab-473b-4828-894f-27ba8c298ace',
 '416d8a84-5fb5-4592-98c2-6595758b9a3e',
 'e4a771ed-27c3-47b0-8eb1-cf909f8d034c',
 'ec6b5ff6-0cfc-4bc6-8e72-b0b413f1d572',
 'f3f93c5d-4ea4-41fe-8257-82fdccd8ce63',
 '43219359-056c-4d79-9ff4-e9ed1fa98659',
 'f12956be-098d-437a-ac23-5dc4890a699f',
 '8e2356ac-745d-46fb-86ce-6f0741b0f6e4',
 '364dfd2f-8774-46ef-b5ff-99c5265904e7',
 '66d8fd70-fd86-4a4c-8b69-9de174d88c95',
 '8d29d7f4-f72a-45ca-86d1-358a8291d785',
 '72851284-f052-4345-8b84-3d2011b9ceb6',
 '66cf2d44-32b8-4d11-b599-1e59abd74aeb',
 '49afa0eb-b060-4932-bd5c-4d0cc9cc97c5',
 'cd62acac-f80e-4640-978e-e083b96d33a4',
 '3102acc0-3fbd-469b-aa90-a7a053f97feb',
 'b1f4b3c1-d5dc-4d8a-9d06-6acd6aa5679e',
 '56a2c296-4f70-49f7-b161-7ec0984819d6',
 '2bd14adb-4d52-45f1-8653-2c5f2904afa8',
 'bae417d1-8eaa-

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
key = userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(google_api_key=key,model="gemini-1.5-flash")

In [10]:

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [14]:
sample_docs = retriever.get_relevant_documents("admission")
for doc in sample_docs:
    print(doc.page_content)

Question: When is the admission deadline?
Answer: The admission deadline is April 30.
Question: When is the admission deadline?
Answer: The admission deadline is April 30.
Question: When is the admission deadline?
Answer: The admission deadline is April 30.
Question: When is the admission deadline?
Answer: The admission deadline is April 30.
Question: When is the admission deadline?
Answer: The admission deadline is April 30.


In [15]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful Faq Bot.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

output_parser = StrOutputParser()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | llm
    | output_parser
)

In [22]:
# response = rag_chain.invoke("""Please summarize Leave No Context Behind:
#                             Efficient Infinite Context Transformers with Infini-attention""")

# response



response = rag_chain.invoke("""What is undergraduate program fees?""")

response

'$15,000 per year'

In [23]:

# Step 7: Set Up Retrieval-Augmented Generation (RAG) Chain
from langchain.chains import RetrievalQA
faq_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever,chain_type="stuff",)

In [25]:
# Step 9: Ask a Question
question = "What courses are offered in Computer Science?"
response = faq_chain.invoke(question)

print("Response:", response)

Response: {'query': 'What courses are offered in Computer Science?', 'result': 'Courses include AI, Data Science, and Web Development.'}
